In [1]:
%%configure
{
    "--conf": "spark.sql.extensions=org.apache.spark.sql.hudi.HoodieSparkSessionExtension",
    "--conf": "spark.serializer=org.apache.spark.serializer.KryoSerializer",
    "--datalake-formats": "hudi"
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,None,pyspark,idle,,,None,


In [2]:
import sys
import boto3

from pyspark.context import SparkContext

from awsglue.context import GlueContext
from awsglue.utils import getResolvedOptions
from awsglue.job import Job

spark_ctx = SparkContext.getOrCreate()
glue_ctx = GlueContext(spark_ctx)
spark_ses = glue_ctx.spark_session

The code failed because of a fatal error. Some things to try: a) Make sure Spark has enough available resources for Jupyter to create a Spark context. b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.   c) Restart the kernel.


In [4]:
import boto3

boto_ses = boto3.session.Session()
sts_client = boto_ses.client("sts")
aws_account_id = sts_client.get_caller_identity()["Account"]
aws_region = boto_ses.region_name

print(f"aws_account_id = {aws_account_id}")
print(f"aws_region = {aws_region}")

aws_account_id = 807388292768
aws_region = us-east-1


In [5]:
%pip install s3pathlib>=2.0.1 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.1 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.27.96 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
from s3pathlib import S3Path, context

context.attach_boto_session(boto_ses)

In [3]:
pdf = spark_ses.createDataFrame(
    [
        ("id-1", "2000", "01", "01", "2000-01-01 00:00:00", 1),
        ("id-2", "2000", "01", "02", "2000-01-02 00:00:00", 2),
        ("id-3", "2000", "01", "03", "2000-01-03 00:00:00", 3),
    ],
    ("id", "year", "month", "day", "ts", "value"),
)
pdf.show()

+----+----+-----+---+-------------------+-----+
|  id|year|month|day|                 ts|value|
+----+----+-----+---+-------------------+-----+
|id-1|2000|   01| 01|2000-01-01 00:00:00|    1|
|id-2|2000|   01| 02|2000-01-02 00:00:00|    2|
|id-3|2000|   01| 03|2000-01-03 00:00:00|    3|
+----+----+-----+---+-------------------+-----+



In [8]:
additional_options={
    "hoodie.table.name": "mytable",
    "hoodie.datasource.write.storage.type": "COPY_ON_WRITE",
    "hoodie.datasource.write.operation": "upsert",
    "hoodie.datasource.write.recordkey.field": "id",
    "hoodie.datasource.write.precombine.field": "ts",
    "hoodie.datasource.write.partitionpath.field": "year,month,day",
    "hoodie.datasource.write.hive_style_partitioning": "true",
    "hoodie.datasource.hive_sync.enable": "true",
    "hoodie.datasource.hive_sync.database": "mydatabase",
    "hoodie.datasource.hive_sync.table": "mytable",
    "hoodie.datasource.hive_sync.partition_fields": "year,month,day",
    "hoodie.datasource.hive_sync.partition_extractor_class": "org.apache.hudi.hive.MultiPartKeysValueExtractor",
    "hoodie.datasource.hive_sync.use_jdbc": "false",
    "hoodie.datasource.hive_sync.mode": "hms",
    "path": f"s3://{aws_account_id}-{aws_region}-data/projects/hudi-poc/databases/mydatabase/mytable"
}
(
    pdf.write.format("hudi")
    .options(**additional_options)
    .mode("overwrite")
    .save()
)

Py4JJavaError: An error occurred while calling o83.save.
: java.lang.ClassNotFoundException: 
Failed to find data source: hudi. Please find packages at
https://spark.apache.org/third-party-projects.html
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedToFindDataSourceError(QueryExecutionErrors.scala:574)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:675)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:725)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:864)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:256)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:247)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	a